In [9]:
import numpy as np
import time
import numba
import cProfile

class YourClass:
    def __init__(self):
        self.number_atoms = np.array([100, 100, 100])  # Example size
        self.box_size = np.array([10.0, 10.0, 10.0])   # Example box size
        self.atoms_positions = np.random.random((np.sum(self.number_atoms), 3)) * 10.0  # Random positions

    def evaluate_rij_matrix_no_numba(self):
        """Matrix of vectors between all particles."""
        Nat = np.sum(self.number_atoms)
        Box = self.box_size[:3]
        rij_matrix = np.zeros((Nat, Nat, 3))
        pos_j = self.atoms_positions
        for Ni in range(Nat-1):
            pos_i = self.atoms_positions[Ni]
            rij_xyz = (np.remainder(pos_i - pos_j + Box/2.0, Box) - Box/2.0)
            rij_matrix[Ni] = rij_xyz
        return rij_matrix

# Function outside of the class for Numba optimization
@numba.njit
def evaluate_rij_matrix_numba(pos_i, pos_j, Box):
    """Matrix of vectors between all particles (Numba optimized)."""
    Nat = pos_i.shape[0]
    rij_matrix = np.zeros((Nat, Nat, 3))
    for Ni in range(Nat-1):
        rij_xyz = (np.remainder(pos_i[Ni] - pos_j + Box/2.0, Box) - Box/2.0)
        rij_matrix[Ni] = rij_xyz
    return rij_matrix

# Create an instance of the class
obj = YourClass()

# Measure time for non-Numba version
start_time = time.time()
rij_matrix_no_numba = obj.evaluate_rij_matrix_no_numba()
end_time = time.time()
print("Execution time without Numba:", end_time - start_time)

# Warm-up Numba function
evaluate_rij_matrix_numba(obj.atoms_positions, obj.atoms_positions, obj.box_size[:3])
# Measure time for Numba version (after warm-up)
start_time = time.time()
rij_matrix_numba = evaluate_rij_matrix_numba(obj.atoms_positions, obj.atoms_positions, obj.box_size[:3])
end_time = time.time()
print("Execution time with Numba (after warm-up):", end_time - start_time)

# Optionally, use cProfile to analyze performance further
# cProfile.run('evaluate_rij_matrix_numba(obj.atoms_positions, obj.atoms_positions, obj.box_size[:3])')


Execution time without Numba: 0.009469747543334961
Execution time with Numba (after warm-up): 0.0017948150634765625


In [11]:
import time

# Original implementation
def potentials_original(epsilon, sigma, r, derivative=False):
    """ LJ potential for interaction between a pair of neutral atoms"""
    if derivative:  # compute the derivative of the Lennard-Jones potential
        return 48 * epsilon * ((sigma / r) ** 12 - 0.5 * (sigma / r) ** 6) / r
    else:  # calculate the Lennard-Jones potential itself
        return 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)

# Refactored implementation
def potentials_refactored(epsilon, sigma, r, derivative=False):
    """Compute the Lennard-Jones potential or its derivative for interaction between a pair of neutral atoms."""
    # Compute the scaled distance for convenience
    scaled_r = sigma / r
    scaled_r_6 = scaled_r ** 6
    scaled_r_12 = scaled_r ** 12
    
    # If derivative is True, return the force (derivative of the potential)
    if derivative:
        force = 48 * epsilon * (scaled_r_12 - 0.5 * scaled_r_6) / r
        return force
    
    # Otherwise, return the potential
    potential = 4 * epsilon * (scaled_r_12 - scaled_r_6)
    return potential

# Set some test parameters
epsilon = 1.0
sigma = 1.0
r_values = [1.5, 2.0, 2.5, 3.0, 3.5, 4.0]  # Different distances to test

# Benchmarking the original implementation
start_time = time.time()
for r in r_values:
    for _ in range(100000):  # Run a large number of iterations to measure time
        potentials_original(epsilon, sigma, r, derivative=True)
        potentials_original(epsilon, sigma, r, derivative=False)
original_time = time.time() - start_time

# Benchmarking the refactored implementation
start_time = time.time()
for r in r_values:
    for _ in range(100000):  # Run a large number of iterations to measure time
        potentials_refactored(epsilon, sigma, r, derivative=True)
        potentials_refactored(epsilon, sigma, r, derivative=False)
refactored_time = time.time() - start_time

# Print the results
print(f"Original implementation time: {original_time:.6f} seconds")
print(f"Refactored implementation time: {refactored_time:.6f} seconds")


Original implementation time: 0.162937 seconds
Refactored implementation time: 0.171991 seconds


In [15]:
import time
import numba

# Original implementation with Numba
@numba.jit(nopython=True)
def potentials_original_numba(epsilon, sigma, r, derivative=False):
    """ LJ potential for interaction between a pair of neutral atoms"""
    if derivative:  # compute the derivative of the Lennard-Jones potential
        return 48 * epsilon * ((sigma / r) ** 12 - 0.5 * (sigma / r) ** 6) / r
    else:  # calculate the Lennard-Jones potential itself
        return 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)

# Refactored implementation with Numba
@numba.jit(nopython=True)
def potentials_refactored_numba(epsilon, sigma, r, derivative=False):
    """Compute the Lennard-Jones potential or its derivative for interaction between a pair of neutral atoms."""
    # Compute the scaled distance for convenience
    scaled_r = sigma / r
    scaled_r_6 = scaled_r ** 6
    scaled_r_12 = scaled_r ** 12
    
    # If derivative is True, return the force (derivative of the potential)
    if derivative:
        force = 48 * epsilon * (scaled_r_12 - 0.5 * scaled_r_6) / r
        return force
    
    # Otherwise, return the potential
    potential = 4 * epsilon * (scaled_r_12 - scaled_r_6)
    return potential

# Set some test parameters
epsilon = 1.0
sigma = 1.0
r_values = [1.5, 2.0, 2.5, 3.0, 3.5, 4.0]  # Different distances to test

# Benchmarking the original implementation with Numba
start_time = time.time()
for r in r_values:
    for _ in range(100000):  # Run a large number of iterations to measure time
        potentials_original_numba(epsilon, sigma, r, derivative=True)
        potentials_original_numba(epsilon, sigma, r, derivative=False)
original_numba_time = time.time() - start_time


potentials_refactored_numba(epsilon, sigma, r, derivative=True)
potentials_refactored_numba(epsilon, sigma, r, derivative=False)
# Benchmarking the refactored implementation with Numba
start_time = time.time()
for r in r_values:
    for _ in range(100000):  # Run a large number of iterations to measure time
        potentials_refactored_numba(epsilon, sigma, r, derivative=True)
        potentials_refactored_numba(epsilon, sigma, r, derivative=False)
refactored_numba_time = time.time() - start_time

# Print the results
print(f"Original implementation (Numba) time: {original_numba_time:.6f} seconds")
print(f"Refactored implementation (Numba) time: {refactored_numba_time:.6f} seconds")


Original implementation (Numba) time: 0.305533 seconds
Refactored implementation (Numba) time: 0.260672 seconds


In [3]:
import numpy as np
import numba

# Sample dummy function for compute_distance and potentials
@numba.njit
def compute_distance(pos_i, pos_j, box_size, only_norm=False):
    rij_xyz = pos_i - pos_j
    rij_xyz = np.remainder(rij_xyz + box_size/2.0, box_size) - box_size/2.0
    rij = np.linalg.norm(rij_xyz)
    if only_norm:
        return rij
    else:
        return rij, rij_xyz

@numba.njit
def potentials(epsilon, sigma, rij, derivative=False):
    # Simplified Lennard-Jones potential calculation
    r = sigma / rij
    force = 48 * epsilon * (r**12 - 0.5 * r**6) / rij  # Derivative w.r.t. rij
    return force

@numba.njit
def compute_force(self, return_vector=True):
    if return_vector:
        force_vector = np.zeros((np.sum(self.number_atoms), 3))
    else:
        force_matrix = np.zeros((np.sum(self.number_atoms), np.sum(self.number_atoms), 3))
    
    for Ni in np.arange(np.sum(self.number_atoms) - 1):
        # Read neighbor list
        neighbor_of_i = self.neighbor_lists[Ni]
        
        # Measure distance
        rij, rij_xyz = compute_distance(self.atoms_positions[Ni], self.atoms_positions[neighbor_of_i], self.box_size, only_norm=False)
        
        # Measure force using information about cross coefficients
        sigma_ij = self.sigma_ij_list[Ni]
        epsilon_ij = self.epsilon_ij_list[Ni]
        fij_xyz = potentials(epsilon_ij, sigma_ij, rij, derivative=True)
        
        if return_vector:
            # Add the contribution to both Ni and its neighbors
            force_vector[Ni] += np.sum((fij_xyz * rij_xyz.T / rij).T, axis=0)
            force_vector[neighbor_of_i] -= (fij_xyz * rij_xyz.T / rij).T
        else:
            # Add the contribution to the matrix
            force_matrix[Ni][neighbor_of_i] += (fij_xyz * rij_xyz.T / rij).T

    if return_vector:
        return force_vector
    else:
        return force_matrix


import time

# Create a dummy object for testing (with necessary attributes)
class YourClass:
    def __init__(self):
        self.number_atoms = np.array([100, 100, 100])  # Example size
        self.box_size = np.array([10.0, 10.0, 10.0])   # Example box size
        self.atoms_positions = np.random.random((np.sum(self.number_atoms), 3)) * 10.0  # Random positions
        self.neighbor_lists = [np.random.randint(0, 100, 10) for _ in range(100)]
        self.sigma_ij_list = np.random.random(100)
        self.epsilon_ij_list = np.random.random(100)
        
    def compute_force_no_numba(self, return_vector=True):
        # Original function without Numba for comparison
        # The function is similar to the Numba version but without `@njit` decorator
        pass  # Original code goes here

# Create an instance of the class
obj = YourClass()

# Measure time for non-Numba version
start_time = time.time()
force_no_numba = obj.compute_force_no_numba(return_vector=True)
end_time = time.time()
print("Execution time without Numba:", end_time - start_time)

# Measure time for Numba version
start_time = time.time()
force_numba = compute_force(return_vector=True)
end_time = time.time()
print("Execution time with Numba:", end_time - start_time)


Execution time without Numba: 3.123283386230469e-05


TypeError: missing argument 'self'